In [37]:
### Import:

In [38]:
import pandas as pd
import numpy as np


### 1. Caricamento del dataset

In [39]:
try:
    df = pd.read_csv("./data/monsters.csv")
    print("✅ Dataset caricato con successo.")
except FileNotFoundError:
    print("❌ Errore: File non trovato. Controlla il percorso.")
    raise


✅ Dataset caricato con successo.


### --- 2. GESTIONE DEI TIRI SALVEZZA (Imputazione basata su regole D&D) ---

In [40]:
stats_map = {
    'strength': 'strength_save',
    'dexterity': 'dexterity_save',
    'constitution': 'constitution_save',
    'intelligence': 'intelligence_save',
    'wisdom': 'wisdom_save',
    'charisma': 'charisma_save'
}

print("--- Elaborazione Tiri Salvezza ---")

for stat_col, save_col in stats_map.items():
    if stat_col in df.columns and save_col in df.columns:
        modifiers = np.floor_divide(df[stat_col] - 10, 2)
        condition = df[save_col].isnull()
        df[save_col] = np.where(condition, modifiers, df[save_col])
        
        if condition.sum() > 0:
            print(f"✅ {save_col}: Imputati {condition.sum()} valori mancanti.")


--- Elaborazione Tiri Salvezza ---


### --- 3. GESTIONE PERCEPTION ---

In [41]:
print("--- Elaborazione Perception ---")
if 'wisdom' in df.columns and 'perception' in df.columns:
    wis_mod = np.floor_divide(df['wisdom'] - 10, 2)
    condition = df['perception'].isnull()
    df['perception'] = np.where(condition, wis_mod, df['perception'])
    print(f"✅ Perception: Imputati {condition.sum()} valori mancanti.")


--- Elaborazione Perception ---
✅ Perception: Imputati 0 valori mancanti.


### --- 4. RIMOZIONE COLONNE RIDONDANTI (Slug & Armor Desc) ---
Rimuoviamo 'slug' perché è un identificativo tecnico ridondante rispetto al nome.
Rimuoviamo 'armor_desc' perché abbiamo già 'armor_class'.

In [42]:
cols_to_drop = ['slug', 'armor_desc']
existing_cols_to_drop = [c for c in cols_to_drop if c in df.columns]

if existing_cols_to_drop:
    df = df.drop(columns=existing_cols_to_drop)
    print(f"\n✅ Colonne rimosse con successo: {existing_cols_to_drop}")
else:
    print("\n⚠️ Colonne 'slug' o 'armor_desc' non trovate (già rimosse?).")



✅ Colonne rimosse con successo: ['slug']


### --- 5. TRASFORMAZIONE HIT DICE ---

In [43]:
print("--- Elaborazione Hit Dice ---")

if 'hit_dice' in df.columns:
    hit_dice_clean = df['hit_dice'].fillna('0d0').astype(str)
    extracted_values = hit_dice_clean.str.split('d', expand=True)[0]
    numeric_values = pd.to_numeric(extracted_values, errors='coerce')
    clean_values = np.nan_to_num(numeric_values, nan=0)
    df['hit_dice_count'] = clean_values.astype(int)
    df = df.drop(columns=['hit_dice'])
    print("✅ Hit Dice convertiti e colonna testuale rimossa.")
else:
    print("⚠️ La colonna 'hit_dice' non è stata trovata.")


--- Elaborazione Hit Dice ---
⚠️ La colonna 'hit_dice' non è stata trovata.


### --- 6. TRASFORMAZIONE SIZE (Ordinal Encoding) ---
Tiny=1 -> Titanic=7

In [44]:
print("--- Elaborazione Size ---")

if 'size' in df.columns:
    size_mapping = {
        'Tiny': 1, 'Small': 2, 'Medium': 3, 'Large': 4, 
        'Huge': 5, 'Gargantuan': 6, 'Titanic': 7
    }
    
    df['size_numeric'] = df['size'].map(size_mapping)
    
    # Gestione valori mancanti (default a Medium=3)
    if df['size_numeric'].isnull().sum() > 0:
        df['size_numeric'] = df['size_numeric'].fillna(3)
        
    df['size_numeric'] = df['size_numeric'].astype(int)
    df = df.drop(columns=['size'])
    print("✅ Size convertita in scala ordinale (1-7).")
else:
     print("⚠️ La colonna 'size' non è stata trovata.")


--- Elaborazione Size ---
⚠️ La colonna 'size' non è stata trovata.


### --- RIORDINAMENTO ---

In [45]:
cols = df.columns.tolist()

# Posiziona hit_dice_count
if 'hit_points' in cols and 'hit_dice_count' in cols:
    cols.remove('hit_dice_count')
    target_idx = cols.index('hit_points')
    cols.insert(target_idx + 1, 'hit_dice_count')

# Posiziona size_numeric
if 'size_numeric' in cols:
    cols.remove('size_numeric')
    # Mettilo come prima colonna se 'name' non c'è, o subito dopo 'name'
    if 'name' in cols:
        target_idx = cols.index('name')
        cols.insert(target_idx + 1, 'size_numeric')
    else:
        cols.insert(0, 'size_numeric')

df = df[cols]


### --- 7. SALVATAGGIO ---

In [46]:
output_path = "./data/monsters.csv"
try:
    df.to_csv(output_path, index=False)
    print(f"\n💾 Salvataggio completato! File aggiornato: {output_path}")
except:
    df.to_csv("monsters_processed.csv", index=False)
    print(f"\n💾 File salvato come: monsters_processed.csv (cartella ./data non trovata)")



💾 Salvataggio completato! File aggiornato: ./data/monsters.csv
